<a href="https://colab.research.google.com/github/anoushka2006/Music-Recommendation-System/blob/main/Music_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Preliminary exploration of the data**

In [ ]:
# General Imports:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import gdown

# for the ML
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Importing packages and dataset
import ast
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from IPython import display
import ipywidgets as widgets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Downloading the data
!wget -O ./spotify_data_urls.csv 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Music%20Recommendation/spotify_data_urls.csv'
data_path = './spotify_data_urls.csv'

In [ ]:
# Load in data
data = pd.read_csv(data_path)
basic_data = data[['Artist','Track','Year','url','Label']]
basic_data.head(10)

In [ ]:
# To understand the parameters and information present
print(data.columns.tolist())
data.info()

In [ ]:
# Analysing different data
# Danceability Histogram
plot1 = plt.figure(1)
plt.hist(data["danceability"], color = 'b')
plt.xlabel('Danceability')
plt.ylabel('Count')
plt.title('Danceability Histogram')
plt.plot()

# Energy Histogram
plot2 = plt.figure(2)
plt.hist(data["energy"], color = 'red')
plt.xlabel('Energy')
plt.ylabel('Count')
plt.title('Energy Histogram')
plt.plot()

# Key Histogram
plot3 = plt.figure(3)
plt.hist(data["key"], color = 'pink')
plt.xlabel('Key')
plt.ylabel('Count')
plt.title('Key Histogram')
plt.plot()

# Loudness Histogram
plot4 = plt.figure(4)
plt.hist(data["loudness"], color = 'green')
plt.xlabel('Loudness')
plt.ylabel('Count')
plt.title('Loudness Histogram')
plt.plot()


### END CODE
plt.show()

In [ ]:
# Subjective features
subjective = data[['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']]

# Objective features
objective = data[['Artist', 'Track', 'Year', 'key', 'loudness', 'mode', 'tempo']]

In [ ]:
# Initialising the X and y data values
X = data[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]
y = data[['Label']]

# The data matrix and label vector
print(X)
print(y)

In [ ]:
# Splitting the data into training and testing
logM = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Initializing the logistic regression model and training it
logM.fit(X_train, y_train)
y_pred = logM.predict(X_test)

# Calculating and printing the score
accuracy = logM.score(X_test, y_test)
print("Accuracy: ", accuracy)

In [ ]:
# To see which feature was the most helpful
coef = logM.coef_
pd.DataFrame(list(zip(X, coef[0])), columns=['feature', 'coef'])

Features with a strong positive coefficient means more of that feature tends to corrrelate to a 'hit' song.
So from the data we can conclude that songs with greater 'danceability' tend to be hit songs. While songs with a ggreater instrumental factor tend to be non-hits.

In [ ]:
# Narrowing the features to only the most useful
# Initalising the X and Y data values
#X_2 = data[['danceability','energy','speechiness','acousticness','instrumentalness']]
X_2 = data[['danceability','instrumentalness']]
y_2 = data[['Label']]

# Splitting the data into training and testing
logM_2 = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Initializing the logistic regression model and training it
logM_2.fit(X_train, y_train)
y_pred = logM_2.predict(X_test)

# Calculating and printing the score
accuracy_2 = logM_2.score(X_test, y_test)
print("Accuracy: ", accuracy_2)

In [ ]:
# Testing and training other models to compare performance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

svc = SVC()
svc.fit(X_train,y_train)
print('Support Vector Classifier score: ',svc.score(X_test,y_test))

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,y_train)
print('KNN score: ',knn.score(X_test,y_test))

gnb = GaussianNB()
gnb.fit(X_train,y_train)
print('Naive Bayes score: ',gnb.score(X_test,y_test))

mlp = MLPClassifier(hidden_layer_sizes=(10,10,5))
mlp.fit(X_train,y_train)
print('Multilayer Perceptron Classifier score: ',mlp.score(X_test,y_test))

All of these models did not perform much better than the multi-feature logistic regression.
Though the MLP neural network classifier does perform decently well due to its ability to consider multiple features to find complex patterns.

#**Introducing new data and Segregation**


In [ ]:
#@title Loading new data
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Music%20Recommendation/data_with_most_lyrics.csv'
music_data_path = './data_with_most_lyrics.csv'

music_data = pd.read_csv(music_data_path)
music_data = music_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'])
music_data['lyrics'] = music_data['lyrics'].str.replace('\n', ' ')
available_songs = music_data["track_name"] + ', ' + music_data["artist_name"]
available_songs = available_songs.tolist()


!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Music%20Recommendation/spotify_data_urls.csv'
past_data_path = './spotify_data_urls.csv'

past_data = pd.read_csv(past_data_path)

class Error(Exception):
    """Base class for other exceptions"""
    pass

class NotSeparableError(Error):
    """Raised when the input value is too small"""
    pass

def fix_genres(row):
  row_new = row['genres'].replace(',', '')
  row_new = row_new.replace("'", "")
  row_new = row_new.replace('"', "")
  if len(row_new)!=0 and row_new[0]=='[':
    return row_new[1:-1]
  return row_new

def find_title_from_index(index):
    return music_data["track_name"][index]
def find_artist_from_index(index):
    return music_data["artist_name"][index]
def find_index_from_title(track_name):
    return music_data.index[music_data.track_name == track_name].values[0]


In [ ]:
# To understand the parameters in the data
music_data.head()

In [ ]:
# A closer look at the genres column
music_data['genres'].head(10)

In [ ]:
# To see if all the values are unique
music_data['genres'].value_counts()

Since a song can have more than one genre, some pre-processing to separate the different genres is necessary before using value_counts. In our dataframe in pandas, the genres are stored as one giant string which needs to be turned into list strings (the genres).

In [ ]:
# Initializer function to separate the genres
def parse_string_into_list(string):
  return string[1:len(string)-1].split(', ')

# To use the stringer parser in the data frame
def separate_values(column):
  mdata = music_data.copy()
  mdata[column] = mdata.apply(lambda row: parse_string_into_list(row[column]), axis=1)
  mdata.head()

  mdata = mdata.explode(column)
  return mdata

In [ ]:
# Printing the separated genres
separated_genres = separate_values('genres')
separated_genres.head()

In [ ]:
# To check the separation
separated_genres['genres'].value_counts()

#**Hit Prediction**

In [ ]:
#@title Chosing a genre
available_genres = separate_values('genres')['genres'].value_counts().index.tolist()
genre = widgets.Dropdown(
    options=available_genres,
    value=None,
    description='Genre:',
    disabled=False,
)
genre

In [ ]:
# Creating the data frame with only tracks from chosen genre
if genre.value!=None:
  genre_tracks = separated_genres.loc[separated_genres['genres'] == genre.value]

In [ ]:
# To order the songs in terms of danceability
danceability_sorted_songs = genre_tracks.sort_values('danceability', ascending=False)
danceability_sorted_songs.head()

In [ ]:
# To order the songs in terms of track popularity
popularity_sorted_songs = genre_tracks.sort_values('track_popularity', ascending=False)
popularity_sorted_songs.head()

In [ ]:
# Settting up logistic regression
X = past_data[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]
y = past_data[['Label']]

# Splitting the data into the training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Training the model
logR = LogisticRegression()
logR.fit(X_train,y_train)


# Creating new X values with music_data
X_new = music_data[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]

# To predict if each song will be a hit or not and adding the prediction to the music_data dataframe
music_data['hit_prediction'] = logR.predict(X_new)

In [ ]:
# To check that the new column for hit_prediction has been created
# 1 if a song is a hit or a 0 if it isn't
music_data.head()

#**Recommending songs based on Top Hits in a Genre**

In [ ]:
#@title To choose your genre
available_genres = separate_values('genres')['genres'].value_counts().index.tolist()
genre = widgets.Combobox(
    placeholder='Choose a Genre',
    options=available_genres,
    description='genre:',
    ensure_option=True,
    disabled=False)
genre

Since the data changed, the genres need to be separated again.

In [ ]:
# Separating the genres
separated_genres = separate_values('genres')

# Segmenting the data based on the chosen genre
if genre.value!=None:
  genre_tracks = separated_genres.loc[separated_genres['genres'] == genre.value]

# Segmenting the data based on hits
  genre_hits = genre_tracks.loc[genre_tracks['hit_prediction'] == 1]

In [ ]:
# Top hits in the chosen genre
pd.set_option("display.max_rows", None, "display.max_columns", None)
genre_hits[['track_name', 'artist_name']].head(10)

#**Content-Based Filtering**

Content-based filtering uses data from one user. Usually, it looks like a recommendation based on something that user previously watched, listened to, or clicked on. We can use this method to make better recommendations that are more specific to an individual user.

In [ ]:
# Features of the data set
music_data.dtypes

In [ ]:
numerical_features = ['danceability', 'track_popularity']
text_features = ['genres']

In [ ]:
# Combines all the features to a string
def combine_features(row):
    # Loop through all of the features and make a string with all of them combined for one row
    combined_row = ''
    for feature in text_features:
      combined_row += str(row[feature])
      combined_row += ' '
    return combined_row[:-1]

In [ ]:
# To use the `combine_features` function
# Getting rid of null values
if 'genres' in text_features:
  music_data['genres'] = music_data['genres'].fillna('')
  music_data['genres'] = music_data.apply(fix_genres, axis=1)
for feature in text_features:
    music_data[feature] = music_data[feature].fillna('')
music_data["combined_features"] = music_data.apply(combine_features,axis=1)

In [ ]:
# Printing
music_data.head()

In [ ]:
# To make a vector out of the combined text features
cv = CountVectorizer()
count_matrix = cv.fit_transform(music_data["combined_features"])

In [ ]:
# Combining the numerical features with the vectorized text features
text_vectors = count_matrix.toarray()
numerical = music_data[numerical_features].to_numpy()

# Scaling numerical values so features like track_popularity and tempo don't outweigh everything else
numerical = (numerical - numerical.min(axis=0)) / (numerical.max(axis=0) - numerical.min(axis=0))
song_vectors = np.concatenate((text_vectors, numerical), axis=1)

print(song_vectors)

#**Checking Similarity**

The most common metric for recommendation systems is **cosine similarity**, so that's what we'll have as our default.

In [ ]:
# To give the similarity scores between all the different songs in the data set using the cosine distance between vectors
def all_similarity(vectors, sim_metric='cosine'):
  if sim_metric == 'cosine':
    return cosine_similarity(vectors)
  else:
    # greater distance means the vectors are less similar, so we negate it
    return -pairwise_distances(vectors, metric=sim_metric)

Each of the numbers represents the cosine(theta) where theta is the angle between the song vectors.If the songs are the same, theta = 0 and cos(0) = 1.0
Cosine is larger for smaller angles so more similar songs have smaller angles and higher numbers.

In [ ]:
sim_matrix = all_similarity(song_vectors, sim_metric='cosine')
print(sim_matrix)

In [ ]:
# shape of the similarity matrix
print(sim_matrix.shape)

It is the size of the total number of songs squared.


To access a value in the matrix, we need to do so with the indices of songs. To get the index, we will call the function `find_index_from_title(song)` that will return the index of a given song.

In [ ]:
# Function to take two songs and give us the similarity score between them
def similarity_score(track1, track2, vectors, metric='cosine'):
  if track1==None or track2==None:
    return None

  similarity_matrix = all_similarity(vectors, metric)

  song1_index = find_index_from_title(track1)
  song2_index = find_index_from_title(track2)
  score = similarity_matrix[song1_index][song2_index]

  return score

In [ ]:
#@title Run to choose the first song
song_1 = widgets.Combobox(
    placeholder='Choose a Song',
    options=available_songs,
    description='song1:',
    ensure_option=True,
    disabled=False)
song_1

In [ ]:
#@title Run to choose the second song
song_2 = widgets.Combobox(
    placeholder='Choose a Song',
    options=available_songs,
    description='song2:',
    ensure_option=True,
    disabled=False)
song_2

In [ ]:
# To process the song choices
if song_1.value!='' and song_2.value!='':
  song1 = music_data['track_name'][available_songs.index(song_1.value)]
  song2 = music_data['track_name'][available_songs.index(song_2.value)]

In [ ]:
score = similarity_score(song1, song2, song_vectors, metric='cosine')
print(score)

The similarity scores will change based on which features are used to calculate the similarity score

In [ ]:
# Trying new features

numerical_features = ['danceability', 'tempo']
text_features = ['genres']

In [ ]:
#@title Run this cell after updating new features in the cell above, then run the similarity_score() cell again
# gets rid of null values
if 'genres' in text_features:
  music_data['genres'] = music_data['genres'].fillna('')
  music_data['genres'] = music_data.apply(fix_genres, axis=1)
for feature in text_features:
    music_data[feature] = music_data[feature].fillna('')
music_data["combined_features"] = music_data.apply(combine_features,axis=1)

# Make the combined vector out of text features
cv = CountVectorizer()
count_matrix = cv.fit_transform(music_data["combined_features"]) # creates a vector out of our combined features

# Adds numerical features to vector
text_vectors = count_matrix.toarray()
numerical = music_data[numerical_features].to_numpy()
# Next line scales numerical values so features like track_popularity and tempo don't outweigh everything else!
numerical = (numerical - numerical.min(axis=0)) / (numerical.max(axis=0) - numerical.min(axis=0))
song_vectors = np.concatenate((text_vectors, numerical), axis=1)

In [ ]:
# the similarity score
if song_1.value!='' and song_2.value!='':
  song1 = music_data['track_name'][available_songs.index(song_1.value)]
  song2 = music_data['track_name'][available_songs.index(song_2.value)]

score = similarity_score(song1, song2, song_vectors, metric='cosine')
print(score)

#**Recommending songs based on content and user selection**

Now that we have information about the similarity between the songs in our data set, we can find the most similar songs and recommend those.

In this section, I will be exploring this using the KNN algorithm (K-Nearest Neighbouts).

In [ ]:
#@title Choose a song
song= widgets.Combobox(
    placeholder='Choose a Song',
    options=available_songs,
    description='song:',
    ensure_option=True,
    disabled=False)
song

In [ ]:
# similarity values with the song against all other songs as vectors
if song.value!='':
  song = music_data['track_name'][available_songs.index(song_1.value)]

similarity_matrix = all_similarity(song_vectors)
song_index = find_index_from_title(song)
song_similarity = similarity_matrix[song_index]

In [ ]:
from threading import current_thread
# Setting a K value
K = 5

#list of integers corresponding to the indices of the max similarity values
song_indices = []

for i in range(0,K):
  current_max = 0
  current_index = 0

  for j in range(0, len(song_similarity)):
    if song_similarity[j] > current_max:
      current_max = song_similarity[j]
      current_index = j
  song_indices.append(current_index)

# using sorting
similar_songs = list(enumerate(song_similarity))
sorted_similar_songs = sorted(similar_songs,key=lambda x:x[1],reverse=True)[1:]
song_indices = [x[0] for x in sorted_similar_songs[:K]]

To see our recommendations, we need convert the indices back into the song titles.

The relevant functions we can call are:
  `find_title_from_index`,
  `find_artist_from_index`

In [ ]:
song_names = []
singers = []

for index in song_indices:
  song_names.append(find_title_from_index(index))
  singers.append(find_artist_from_index(index))

In [ ]:
# to find the k most similar songs
def k_most_similar_songs(song, song_vectors, K):
  similarity_matrix = all_similarity(song_vectors)
  song_index = find_index_from_title(song)
  song_similarity = similarity_matrix[song_index]

  song_indices = []

  for i in range(0, K):
    current_max = 0
    current_index = None
    for j in range(len(song_similarity)):
      if song_similarity[j] > current_max and j not in song_indices and j != song_index:
        current_max = song_similarity[j]
        current_index = j
    song_indices.append(current_index)

  song_names = []
  singers = []
  for index in song_indices:
    song_names.append(find_title_from_index(index))
    singers.append(find_artist_from_index(index))
  return song_names, singers

In [ ]:
#@title Choose a song
song = widgets.Combobox(
    placeholder='Choose a Song',
    options=available_songs,
    description='Song:',
    ensure_option=True,
    disabled=False)
song

In [ ]:
# to get the recommended songs
song_name = music_data['track_name'][available_songs.index(song.value)]
similar_songs, similar_artists = k_most_similar_songs(song_name, song_vectors, 10)
print(similar_songs)
print(similar_artists)

We made two types of recommendation systems: one where we recommend popular songs by genre and one where we're recommending songs based on a song that the user has previously listened to.

#**Streamlit**

In [ ]:
#@title Setting up Streamlit
print("Setting up Streamlit...")
#print("Don't worry if you see ERROR here!")
!pip -q install streamlit
!pip -q install pyngrok
from pyngrok import ngrok

!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Music%20Recommendation/data_with_most_lyrics.csv'
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Music%20Recommendation/spotify_data_urls.csv'

def launch_website():
  print ("Click this link to try your web app:")
  print ("If you get an error, wait 5 seconds and click again!")
  if (ngrok.get_tunnels() != None):
    ngrok.kill()
  public_url = ngrok.connect()
  print (public_url)
  !streamlit run --server.port 80 app.py >/dev/null

print("Done!")


In [ ]:
!ngrok authtoken 2RExST0RbuRw41Umv2KdN9GZfQ8_4rWvqSZ1ti7zo93sLjNV9

**Customising the app**

In [ ]:
%%writefile header.py
import streamlit as st
import streamlit.components.v1 as components

def create_header():
  st.title("Music Recommendation System")
  st.header("By Anoushka Chaudhury")
  st.subheader("This recommender system ...")

In [ ]:
%%writefile response.py
import streamlit as st
import streamlit.components.v1 as components

def get_app_response(recommended_songs, recommended_artists):
  if recommended_songs is not None:
    for i in range(len(recommended_songs)):
      st.write(recommended_songs[i] + ' by ' + recommended_artists[i])

The previous code written needs to be consolidated into functions so they can be used later.

I started with a function that does popular recommendations (recommendations by top hits in the genre)

We have a function outline below that takes in the chosen genre, the number of songs we want to give the user (`K`), and an optional parameter called `method` defaulted to "popularity". This can be used to switch between recommending by popularity value (using "popularity") or recommending by top hit prediction (using "hit_prediction").

This function should return the K recommended songs and their artists as two separate lists.

**Popular Recs in Streamlit**

In [ ]:
#@title Popular and Hit-based Recommendation
%%writefile helper.py
import streamlit as st
import streamlit.components.v1 as components
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Music data
music_data = pd.read_csv('data_with_most_lyrics.csv')
music_data = music_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
music_data['lyrics'] = music_data['lyrics'].str.replace('\n', ' ')
available_songs = music_data["track_name"] + ', ' + music_data["artist_name"]
available_songs = available_songs.tolist()

# Past music data
past_data = pd.read_csv('spotify_data_urls.csv')

def find_title_from_index(index):
    return music_data["track_name"][index]
def find_artist_from_index(index):
    return music_data["artist_name"][index]
def find_index_from_title(track_name):
    return music_data.index[music_data.track_name == track_name].values[0]
# end imports



def top_song_recs(genre, K, method='track_popularity'):
  # separating genre values
  separated_genres = separate_values('genres')

  # segmenting data based on chosen genre
  genre_tracks = None
  if genre!=None:
    genre_tracks = separated_genres.loc[separated_genres['genres'] == genre]

    if method == 'hit_prediction':
      X = past_data[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]
      y = past_data[['Label']]

      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

      lr = LogisticRegression()
      lr.fit(X_train,y_train)

      X_new = genre_tracks[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]
      genre_tracks['hit_prediction'] = lr.predict(X_new)

      genre_hits = genre_tracks.loc[genre_tracks['hit_prediction'] == 1][:K]
    else:
      genre_hits = genre_tracks.sort_values('track_popularity', ascending=False)[:K]

    return genre_hits['track_name'].tolist(), genre_hits['artist_name'].tolist()
  return None, None



def parse_string_into_list(string):
  return string[1:len(string)-1].split(', ')

def separate_values(column):
  mdata = music_data.copy()
  mdata[column] = mdata.apply(lambda row: parse_string_into_list(row[column]), axis=1)
  mdata.head()

  mdata = mdata.explode(column)
  return mdata


**Content-Based Recommendation in Streamlit**

We are going to do the same thing for content-based recommendation using the `k_most_similar_songs` function.

In [ ]:
#@title Content-Based Recommendation
%%writefile contentbased.py
import streamlit as st
import streamlit.components.v1 as components
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Music data
music_data = pd.read_csv('data_with_most_lyrics.csv')
music_data = music_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
music_data['lyrics'] = music_data['lyrics'].str.replace('\n', ' ')
available_songs = music_data["track_name"] + ', ' + music_data["artist_name"]
available_songs = available_songs.tolist()

# Past music data
past_data = pd.read_csv('spotify_data_urls.csv')

def find_title_from_index(index):
    return music_data["track_name"][index]
def find_artist_from_index(index):
    return music_data["artist_name"][index]
def find_index_from_title(track_name):
    return music_data.index[music_data.track_name == track_name].values[0]
# end imports



def k_most_similar_songs(song, song_vectors, K):
  similarity_matrix = all_similarity(song_vectors)
  song_index = find_index_from_title(song)
  song_similarity = similarity_matrix[song_index]

  song_indices = []
  for i in range(0, K):
    current_max = 0
    current_index = None
    for j in range(len(song_similarity)):
      if song_similarity[j] > current_max and j not in song_indices and j != song_index:
        current_max = song_similarity[j]
        current_index = j
    song_indices.append(current_index)

  song_names = []
  singers = []
  for index in song_indices:
    song_names.append(find_title_from_index(index))
    singers.append(find_artist_from_index(index))
  return song_names, singers



def combine_features(row, text_features=[]):
    combined_row = ''
    for feature in text_features:
      combined_row += str(row[feature])
      combined_row += ' '
    return combined_row[:-1]

def data_formatting(text_features, numerical_features):
  if 'genres' in text_features:
    music_data['genres'] = music_data['genres'].fillna('')
    # music_data['genres'] = music_data.apply(fix_genres, axis=1)
  for feature in text_features:
      music_data[feature] = music_data[feature].fillna('')
  music_data["combined_features"] = music_data.apply(lambda row: combine_features(row, text_features=text_features),axis=1)

  cv = CountVectorizer()
  count_matrix = cv.fit_transform(music_data["combined_features"]) # creates a vector out of our combined features

  text_vectors = count_matrix.toarray()
  numerical = music_data[numerical_features].to_numpy()
  song_vectors = np.concatenate((text_vectors, numerical), axis=1)
  return song_vectors


def all_similarity(vectors, sim_metric='cosine'):
  if sim_metric == 'cosine':
    return cosine_similarity(vectors)
  else:
    # greater distance means the vectors are less similar, so we negate it
    return -pairwise_distances(vectors, metric=sim_metric)

**Combining all the functions and setting up a web interface**

In [ ]:
#@title Putting everything together
%%writefile app.py
import streamlit as st
import streamlit.components.v1 as components

import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
from header import *
from response import *
from helper import *
from contentbased import *

create_header()

def get_song_recs(rec_method, text_features, numerical_features, song, genre, K):
  if rec_method == "Popular Songs":
    return top_song_recs(genre, K)
  elif rec_method == "Predicted Top Hits":
    return top_song_recs(genre, K, method='hit_prediction')
  else:
    if song is not None and song is not []:
      song_vectors = data_formatting(text_features, numerical_features)
      return k_most_similar_songs(song, song_vectors, K)
    return None, None


choice = st.radio("Choose recommendation method: ", ["Popular Songs", "Predicted Top Hits", "Content-Based Recommendation"])
if choice == "Content-Based Recommendation":
  left_column, m1, m2 = st.columns((1,1,1))
  with left_column:
    k = st.slider("Pick a value for K", 1, 20)

  genre = None
  with m1:
    avail_text_feat = ['artist_name', 'track_name', 'album', 'genres', 'lyrics']
    text_features = st.multiselect("Pick your text features!", avail_text_feat)
  with m2:
    avail_num_feat = ['track_popularity', 'year', 'artist_popularity', 'artist_followers', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness','acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature']
    numerical_features = st.multiselect("Pick your numerical features!", avail_num_feat)

  select = st.multiselect("Pick a song!", available_songs, default = None)
  if select == []:
    song = None
  else:
    song = music_data['track_name'][available_songs.index(select[0])]
else:
  left_column, right_column = st.columns(2)
  with left_column:
    k = st.slider("Pick how many songs you want to see!", 1, 20)
  with right_column:
    available_genres = separate_values('genres')['genres'].value_counts().index.tolist()
    select = st.multiselect("Pick a genre!", available_genres)
    if select == []:
      genre = None
    else:
      genre = select[0]
  song = None
  text_features, numerical_features = None, None

recs, artists = get_song_recs(choice, text_features, numerical_features, song, genre, k)
get_app_response(recs, artists)

**Note:** Predicted Top Hits and Content-Based Recommendation may take a LONG time to run

In [ ]:
launch_website()

Click this link to try your web app:
If you get an error, wait 5 seconds and click again!
NgrokTunnel: "https://5219-34-75-168-187.ngrok-free.app" -> "http://localhost:80"
2024-07-28 08:18:03.679455: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 08:18:03.679523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 08:18:03.682045: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 08:18:03.696546: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To ena